In [27]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [40]:
# ✅ 데이터 로드
file_path = "final_dataset.csv"
df = pd.read_csv(file_path)

# ✅ 결측치 처리
df.fillna(0, inplace=True)  # 모든 결측치를 0으로 채움

# 🔥 원핫 인코딩 (체형, 성별)
categorical_features = ["체형", "성별"]
ohe = OneHotEncoder(sparse_output=False, drop="first")  # <- Warning 해결 (sparse → sparse_output)
ohe_encoded = ohe.fit_transform(df[categorical_features])
ohe_df = pd.DataFrame(ohe_encoded, columns=ohe.get_feature_names_out())

# ✅ 기존 카테고리컬 컬럼 제거 후 원핫 인코딩된 데이터 결합
df.drop(columns=categorical_features, inplace=True)
df = pd.concat([df, ohe_df], axis=1)

# 🔥 **타겟 변수 변환 (문자 → 숫자)**
label_encoder = LabelEncoder()
df["필요식단"] = label_encoder.fit_transform(df["필요식단"])

# ✅ 학습 데이터 준비
X = df.drop(columns=["필요식단"])  # 입력 변수
y = df["필요식단"]  # 숫자로 변환된 타겟 변수

# ✅ 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔥 TF-IDF 변환기 생성
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

# 🔥 '식품명(한글)' 컬럼을 TF-IDF로 변환
X_train_tfidf = tfidf.fit_transform(X_train["식품명(한글)"].astype(str))
X_test_tfidf = tfidf.transform(X_test["식품명(한글)"].astype(str))

# ✅ 기존 데이터에서 '식품명(한글)' 컬럼 제거 후, 숫자형 데이터로 변환
X_train = X_train.drop(columns=["식품명(한글)"]).apply(pd.to_numeric, errors="coerce").fillna(0)
X_test = X_test.drop(columns=["식품명(한글)"]).apply(pd.to_numeric, errors="coerce").fillna(0)

# ✅ TF-IDF 데이터를 기존 데이터와 결합
X_train = hstack([X_train.values, X_train_tfidf])
X_test = hstack([X_test.values, X_test_tfidf])

# ✅ XGBoost 모델 학습 (enable_categorical=True 추가)
model = xgb.XGBClassifier(enable_categorical=True, use_label_encoder=False, eval_metric="mlogloss",
                         max_depth=5, learning_rate=0.1, n_estimators=500)
# model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=500)

# ✅ XGBoost 모델 학습
model.fit(X_train, y_train)

# ✅ 예측 및 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ 모델 정확도: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

# ✅ 모델 저장 (원하면 주석 해제)
# model.save_model("xgboost_diet_model.json")

# # 🔥 (선택) 예측값을 다시 원래 문자열 라벨로 변환
# decoded_y_pred = label_encoder.inverse_transform(y_pred)
# print("🔍 예측된 식단 카테고리 예시:", decoded_y_pred[:5])  # 일부만 출력

C:\Users\fn45\anaconda3\envs\myenv\lib\site-packages\xgboost\core.py:158: UserWarning: [15:34:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ 모델 정확도: 0.5775
              precision    recall  f1-score   support

           0       0.25      0.22      0.24       673
           1       0.26      0.28      0.27       675
           2       1.00      1.00      1.00       667
           3       1.00      1.00      1.00       681
           4       1.00      1.00      1.00       687
           5       1.00      1.00      1.00      1269
           6       0.23      0.20      0.21       678
           7       0.29      0.32      0.30       682
           8       0.26      0.24      0.25       652
           9       1.00      1.00      1.00       687
          10       1.00      1.00      1.00       674
          11       0.29      0.37      0.33       638
          12       0.22      0.17      0.19       688
          13       0.24      0.22      0.23       686
          14       0.29      0.32      0.30       669
          15       0.24      0.21      0.23       672
          16       1.00      1.00      1.00       629
          

In [41]:
print(y_train[:10])  # 라벨 데이터 일부 출력
print(set(y_train))  # 라벨에 어떤 값들이 있는지 확인


3508     18
821      14
2532     14
2706     14
54573     5
36228    10
26466    15
35230    10
59716    12
17377    16
Name: 필요식단, dtype: int32
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [42]:
print(y_train.value_counts())

필요식단
5     5379
16    2695
19    2689
18    2687
11    2686
8     2672
2     2657
14    2655
15    2652
0     2651
10    2650
1     2649
6     2646
3     2643
7     2642
17    2642
13    2638
9     2637
4     2637
12    2636
Name: count, dtype: int64


In [43]:
import numpy as np

# 사용자가 입력할 체형과 성별 (예: 모래시계형 & 여성)
user_input = pd.DataFrame({
    "체형_모래시계형": [1],  # 원핫 인코딩된 체형 값
    "체형_사과형": [0],
    "체형_배형": [0],
    "체형_엉덩이형": [0],
    "체형_상체형": [0],
    "체형_하체형": [0],
    "체형_표준체형": [0],
    "성별_여성": [1],  # 원핫 인코딩된 성별 값
})

# TF-IDF 벡터화된 식품명 데이터 추가 (이전 데이터에서 평균값으로 대체)
text_features_avg = np.zeros((1, X_train.shape[1] - user_input.shape[1]))  # 2D로 변환

# 2D 배열끼리 결합
user_input_extended = np.hstack([user_input, text_features_avg])
# 예측 수행
predicted_diet = model.predict(user_input_extended)

# 예측된 숫자를 원래 라벨(필요식단 이름)로 변환
recommended_diet = label_encoder.inverse_transform(predicted_diet)

print("✅ 추천 식단:", recommended_diet[0])


✅ 추천 식단: 단백질 보충 & 저탄수화물


In [44]:
# ✅ 중복 제거 후, 상위 10개 식품만 선택
recommended_foods = recommended_foods.drop_duplicates(subset=["식품명(한글)"]).head(10)

# ✅ 영양소 값이 너무 작으면 조정 (필요 시)
if recommended_foods["단백질"].max() < 10:  
    recommended_foods[["단백질", "탄수화물", "지방", "칼로리"]] *= 100

# ✅ 결과 출력
if recommended_foods.empty:
    print("\n⚠️ 해당 식단에 맞는 추천 식품이 없습니다.")
else:
    print("\n🔹 추천 식품 리스트:")
    print(recommended_foods)



🔹 추천 식품 리스트:
      식품명(한글)        단백질       탄수화물         지방        칼로리
13296     가리비  80.737403   7.322527  33.470122  33.918297
13299      가지   0.413765   0.366223   0.053404   0.345224
13302    가지구이   5.735354   0.192749   4.778234   3.164557
13305     가츠동   5.182302   0.285911   7.335994   4.415995
13308      간장   2.572716   0.488939   0.011243   0.762371
13311    갈비구이  37.279803   2.184486  12.086121  12.802071
13314     갈비찜   6.964359   0.256998   4.216089   3.020713
13317     갈비탕   6.964359   0.256998   4.216089   3.020713
13320       감   0.237608   1.194400   0.053404   1.006904
13323    감귤주스   2.072921  10.905727   0.126483   9.853280
